# Bottle OCR Pipeline Test Notebook (Compliance Verified)

This notebook iterates through all images in `data/test/images` and runs the `BottlePipeline` in **Compliance Mode**.
It reports:
*   **Validation Status**: (Verified Match / Mismatch / Unverified)
*   **Enrichment Data**: Manufacturer, Active Ingredients, Warnings
*   **Raw findings**: Barcode, Layout-Aware OCR Candidates

In [ ]:
import os
import glob
import cv2
import matplotlib.pyplot as plt
from bottle_pipeline import BottlePipeline

# Initialize Pipeline
pipeline = BottlePipeline(ner_model="en_core_web_sm") 

In [ ]:
def show_image(img_path):
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(6, 6))
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()

test_dir = "data/test/images"
image_files = glob.glob(os.path.join(test_dir, "*.jpg"))

print(f"Found {len(image_files)} test images.")

In [ ]:
# Run on all images
for img_path in image_files:
    try:
        result = pipeline.run(img_path)
        
        # --- Compliance Report ---
        print(f"\n{'='*20} COMPLIANCE REPORT: {os.path.basename(img_path)} {'='*20}")
        
        # 1. Validation Status
        status = result.get('validation_status', 'Unknown')
        print(f"STATUS: {status.upper()}")
        
        # 2. Identification
        fda = result.get('fda_data')
        if fda:
            print(f"Drug Identified: {fda['brand_name']}")
            print(f"Manufacturer:  {fda['manufacturer']}")
            print(f"Composition:   {fda['active_ingredients']}")
        else:
            print("Drug Identified: None")
            
        # 3. Raw Evidence
        print("-"*40)
        bc = [b['data'] for b in result['barcodes']] if result['barcodes'] else 'None'
        print(f"Barcode Evidence: {bc}")
        
        # Show Candidates
        cands = result.get('drug_candidates', [])
        print(f"Candidates Found (Layout+Regex): {cands}")
        
        show_image(img_path)

    except Exception as e:
        print(f"Error: {e}")
    
    print("="*70)